In [ ]:
import pandas as pd
from IPython.core.display import display
from sklearn.model_selection import KFold, TimeSeriesSplit

from modules.datasets import make_time_series_data

# Select CV method.
cv_method = "KFold" # KFold, TimeSeriesSplit

# Generatesample data.
n_samples = 9
n_features = 10

x = make_time_series_data(
    n_samples=n_samples,
    n_features=n_features,
    random_state=0,
    start="2021-01-01 00:00:00",
)

y = pd.DataFrame(
    0.5 * x.iloc[:, 0] + 0.4 * x.iloc[:, 1] - 0.3 * x.iloc[:, 2], columns=["target"]
)

df = pd.concat([y, x], axis=1)

n_cv = 3
random_state = 100

if cv_method == "KFold":
    cv = KFold(n_splits=n_cv, shuffle=True, random_state=random_state)
elif cv_method == "TimeSeriesSplit":
    cv = TimeSeriesSplit(n_splits=n_cv)
else:
    raise ValueError("CV method must be 'KFlod' or 'TimeSeriesSplit'.")

# Preapre CV data
data_scaled_cv = []
for i_cv, (train_index, test_index) in enumerate(cv.split(df)):
    data_scaled_cv.append({"train": {}, "test": {}})
    data_scaled_cv[i_cv]["train"]["index"] = train_index
    data_scaled_cv[i_cv]["test"]["index"] = test_index

    # Change traind data background color to grey
    train_slice = pd.IndexSlice[df.index[train_index], :]
    test_slice = pd.IndexSlice[df.index[test_index], :]
    df_color = df.copy().style.set_properties(
        **{"background-color": "#666"}, subset=train_slice
    ).set_properties(
        **{"background-color": "#000"}, subset=test_slice
    )
    display(df_color)
